# GREATSCHOOLS.ORG API XML to CSV ETL Process
## This is a quick Python program to extract, parse API call responses from GreatSchool API (XML)
## Output is entered into a Pandas DataFrame, then ETL Processed:
#### 1- remove unused columns
#### 2 - normalize column names
#### 3 - filter types to public school only
#### 4 - add calculated columns for statistical analysis

In [1]:
# Import Dependency
import requests
import pandas as pd
from config import api_key

from io import StringIO, BytesIO
from lxml import etree as et

# import Zip code list for greater Austin-Round Rock Metro area
csv_reader = pd.read_csv('zip-list.csv')
zip_df = pd.DataFrame(csv_reader)
zipList = list(zip_df.Zip)

In [2]:
# API prep
url = 'https://api.greatschools.org/schools/nearby?'
queryURL = url + 'key=' + api_key + '&state=TX' + '&zip='

# print(queryURL)   

In [3]:
# Process API Call response from XML into DataFrame
df1 = pd.DataFrame()
#tree = et.parse(xml)
#root = ElementTree(xml.content)

# build a series
d = {}

for zip in zipList:
    queryURL1 = queryURL + str(zip)
    # print('Processing: ' + queryURL1)
    xml = requests.get(queryURL1)
    tree = et.fromstring(xml.content)
    for child in tree:
        for children in child:
            d[str(children.tag)] = str(children.text)
        df1 = df1.append(d, ignore_index=True)

df1.head()

,address,city,distance,district,districtId,districtNCESId,enrollment,fax,gradeRange,gsId,...,ncesId,overviewLink,parentRating,phone,ratingsLink,reviewsLink,schoolStatsLink,state,type,website
0,"304 College Avenue, \nFlorence, TX 76527",Florence,1.69,Florence Independent School District,381,4819320,486,(254) 793-3158,PK-5,2512,...,481932001790,https://www.greatschools.org/texas/florence/25...,4,(254) 793-2497,https://www.greatschools.org/texas/florence/25...,https://www.greatschools.org/texas/florence/25...,https://www.greatschools.org/texas/florence/25...,TX,public,http://www.florence.k12.tx.us/Elem.htm
1,"401 Farm to Market Road 970, \nFlorence, TX 7...",Florence,2.44,Florence Independent School District,381,4819320,308,(254) 793-3784,9-12,2513,...,481932001791,https://www.greatschools.org/texas/florence/25...,3,(254) 793-2495,https://www.greatschools.org/texas/florence/25...,https://www.greatschools.org/texas/florence/25...,https://www.greatschools.org/texas/florence/25...,TX,public,http://hs.florenceisd.net/
2,"718 South Patterson Avenue, \nFlorence, TX 76527",Florence,2.59,Florence Independent School District,381,4819320,231,(254) 793-3054,6-8,2516,...,481932021552,https://www.greatschools.org/texas/florence/25...,4,(254) 793-2504,https://www.greatschools.org/texas/florence/25...,https://www.greatschools.org/texas/florence/25...,https://www.greatschools.org/texas/florence/25...,TX,public,http://www.ms.florenceisd.net/
3,"P.O. Box 248, \nGranger, TX 76530",Granger,0.0,Florence Independent School District,381,4819320,231,(512) 859-2649,PK-6,7299,...,01323257,https://www.greatschools.org/texas/granger/729...,4,(512) 859-2927,https://www.greatschools.org/texas/granger/729...,https://www.greatschools.org/texas/granger/729...,https://www.greatschools.org/texas/granger/729...,TX,private,None
4,"P O Box 578, \nGranger, TX 76530",Granger,1.74,Granger Independent School District,442,4821590,406,(512) 859-2446,PK-12,3063,...,482159002173,https://www.greatschools.org/texas/granger/306...,4,(512) 859-2173,https://www.greatschools.org/texas/granger/306...,https://www.greatschools.org/texas/granger/306...,https://www.greatschools.org/texas/granger/306...,TX,public,www.esc13.net/granger


In [4]:
# drop duplicates, keep one row
df1.drop_duplicates(subset='name', keep='last', inplace=True)
df1 = df1.dropna()

In [5]:
df2 = pd.DataFrame()
#df2 = df1.loc[df1['type'] == 'public']

df2 = df1

df2.reset_index(drop=True, inplace=True)

In [6]:
# check record integrity
df2.describe().T

,count,unique,top,freq
address,648,588,"8509 Farm to Market 969, \nAustin, TX 78724",7
city,648,41,Austin,347
distance,648,298,3.44,9
district,648,58,Austin Independent School District,182
districtId,648,58,115,182
districtNCESId,648,57,4808940,182
enrollment,648,444,40,7
fax,648,515,None,99
gradeRange,648,62,PK-5,206
gsId,648,648,4340,1


In [7]:
# list all columns
list(df2)

['address',
 'city',
 'distance',
 'district',
 'districtId',
 'districtNCESId',
 'enrollment',
 'fax',
 'gradeRange',
 'gsId',
 'gsRating',
 'lat',
 'lon',
 'name',
 'ncesId',
 'overviewLink',
 'parentRating',
 'phone',
 'ratingsLink',
 'reviewsLink',
 'schoolStatsLink',
 'state',
 'type',
 'website']

In [8]:
# check record consistency
df2.count()

address            648
city               648
distance           648
district           648
districtId         648
districtNCESId     648
enrollment         648
fax                648
gradeRange         648
gsId               648
gsRating           648
lat                648
lon                648
name               648
ncesId             648
overviewLink       648
parentRating       648
phone              648
ratingsLink        648
reviewsLink        648
schoolStatsLink    648
state              648
type               648
website            648
dtype: int64

In [9]:
# final report selected columns and calculated column
df3 = pd.DataFrame()
df3['ID'] = df2['gsId']
df3['Name'] = df2['name']
df3['Type'] = df2['type']
df3['City'] = df2['city']
df3['Zip'] = df2['address'].str[-5:]
df3['Grade'] = df2['gradeRange']
df3['Rating1'] = df2['gsRating']
df3['Rating2'] = df2['parentRating']
df3['RatingAll'] = pd.to_numeric(df3['Rating1'])*1.1 + pd.to_numeric(df3['Rating2'])*0.5
df3['lat'] = df2['lat']
df3['lon'] = df2['lon']
df3['Enrollment'] = df2['enrollment']

df3.reset_index(inplace=True)

df3.to_csv('GreatSchools_dataset.csv',index=False)



In [10]:
df3.head()

,index,ID,Name,Type,City,Zip,Grade,Rating1,Rating2,RatingAll,lat,lon,Enrollment
0,0,2512,Florence Elementary School,public,Florence,76527,PK-5,4,4,6.4,30.84529,-97.79381,486
1,1,2513,Florence High School,public,Florence,76527,9-12,4,3,5.9,30.832527,-97.798096,308
2,2,2516,Florence Middle School,public,Florence,76527,6-8,4,4,6.4,30.83157,-97.79259,231
3,3,7299,Sts. Cyril and Methodius Catholic,private,Granger,76530,PK-6,4,4,6.4,30.707022,-97.416504,231
4,4,3063,Granger School,public,Granger,76530,PK-12,4,4,6.4,30.717693,-97.44278,406


In [12]:
!jupyter nbconvert --to Python GreatSchools-API-XML-to-Dataframe-ETL.ipynb

[NbConvertApp] Converting notebook GreatSchools-API-XML-to-Dataframe-ETL.ipynb to Python
[NbConvertApp] Writing 2499 bytes to GreatSchools-API-XML-to-Dataframe-ETL.py
